# Multi label Classification using Scikit-multilearn
---


importing all the modules

In [36]:
from sklearn.preprocessing import LabelBinarizer , LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest , chi2 , f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss , hamming_loss
from sklearn.metrics import accuracy_score
from skmultilearn.adapt import MLkNN
from skmultilearn.problem_transform import ClassifierChain
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer

In [19]:
mobility_mode = pd.read_csv('/kaggle/input/mobility/mobility_mode.csv')

### how to handle overlapping class
# multi label classification
alter_col =['Walk', 'Train/Subway', 'Bus', 'Ferry', 'Access-A-Ride']

#use selectKBest from feature_selection
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html
X = mobility_mode.loc[:,~mobility_mode.columns.isin(['Mode','qSURVEYZONE','qZIPHOME','qBOROUGH','Disability_DailyActivity','qEMPLOYMENTCODE']+alter_col)]
### how to handle overlapping class
# y = mobility_mode.loc[:,'Walk']
y_multi = mobility_mode.loc[:,mobility_mode.columns.isin(alter_col)]

In [21]:
oneHot = ['qRACECODE','qMARRIED','qRENT','qEMPLOYMENT','qINDUSTRY','qWORKLOCATION','Welfare']
ordinal = ['qAGECODE','qEDUCATION','qINCOME','qTRIPPLANAPP','gFREIGHT1b_qFREIGHT1_mA','gFREIGHT1b_qFREIGHT2_mA','gFREIGHT1b_qFREIGHT3_mA',
           'gFREIGHT1b_qFREIGHT4_mA','qAUTOVEHICLEFAM','qAUTOVEHICLEWILL','qAUTOVEHICLESAFE','qBUILDINGb','qCHILDREN','Freight_Usage','qWORKFH',
           'Disability_Walking','Disability_Mental','Disability_Seeing','Disability_Hearing','Membership_CarSharing','qCITIBIKE','qSMARTPHONE',
           'qCARACCESS','qLICENSE','qGENDER']

qAGECODE_cate = ['18-24','25-34','35-44','45-54','55-64','65 or older']
qEDUCATION_cate = ['No high school','Some high school','High school graduate or equivalent (i.e., GED)',
                   'Associate degree (i.e., AA, AS)','Some college but degree not received or in progress',
                   'Bachelor\'s degree (i.e., BA, BS, AB)','Graduate degree (i.e., Master\'s, Professional, Doctorate)']
qINCOME_cate = ['Less than $14,999','$15,000 - $24,999','$25,000 - $34,999','$35,000 - $49,999','$50,000 - $74,999',
                '$75,000 - $99,999','$100,000 - $149,999','$150,000-$199,999','$200,000 and above']
qTRIPPLANAPP_cate = ['Less than a few times a year','A few times a year','Less than once a month','Once a month','A few times a month',
                     'Once a week','Several times a week','Daily']
gFREIGHT1b_qFREIGHT1_mA_cate = ['Not applied','Less than a few times','A few times a year','A few times a month','Several times a week','Daily']
gFREIGHT1b_qFREIGHT2_mA_cate = gFREIGHT1b_qFREIGHT1_mA_cate
gFREIGHT1b_qFREIGHT3_mA_cate = gFREIGHT1b_qFREIGHT1_mA_cate
gFREIGHT1b_qFREIGHT4_mA_cate = gFREIGHT1b_qFREIGHT1_mA_cate
qAUTOVEHICLEFAM_cate = ['Not at all familiar','Not very familiar','Somewhat familiar','Very familiar']
qAUTOVEHICLEWILL_cate = ['Not at all willing','Not very willing','Somewhat willing','Very willing']
qAUTOVEHICLESAFE_cate = ['Don\'t Know','Much less safe than standard vehicles','Somewhat less safe than standard vehicles',
                         'Neither more nor less safe than standard vehicles','Somewhat safer than standard vehicles',
                         'Much safer than standard vehicles']
qBUILDINGb_cate = ['A building with 2 to 9 apartments','A building with 10 to 49 apartments','A building with 50 or more apartments',
                   'A one family house attached to one or more houses','A one family house detached from any other house']
qCHILDREN_cate = ['None','One','Two','Three','Four or more']
Freight_Usage_cate = ['Dont know','Less than a few times','A few times a year','A few times a month','Several times a week','Daily']
qWORKFH_cate = ['Yes, work from home','No, do not work from home']
Disability_Walking_cate = ['Yes','No']
Disability_Mental_cate = ['Yes','No']
Disability_Seeing_cate = ['Yes','No']
Disability_Hearing_cate = ['Yes','No']
Membership_CarSharing_cate = ['Yes','No']
qCITIBIKE_cate = ['Yes','No']
qSMARTPHONE_cate = ['Yes','No']
qCARACCESS_cate = ['Yes','No']
qLICENSE_cate = qCARACCESS_cate
qGENDER_cate = ['Female', 'Male']
categories = [qAGECODE_cate,qEDUCATION_cate,qINCOME_cate,qTRIPPLANAPP_cate,gFREIGHT1b_qFREIGHT1_mA_cate,gFREIGHT1b_qFREIGHT2_mA_cate,
              gFREIGHT1b_qFREIGHT3_mA_cate,gFREIGHT1b_qFREIGHT4_mA_cate,qAUTOVEHICLEFAM_cate,qAUTOVEHICLEWILL_cate,qAUTOVEHICLESAFE_cate,
              qBUILDINGb_cate,qCHILDREN_cate,Freight_Usage_cate,qWORKFH_cate,Disability_Walking_cate,Disability_Mental_cate,
              Disability_Seeing_cate,Disability_Hearing_cate,Membership_CarSharing_cate,qCITIBIKE_cate,qSMARTPHONE_cate,
              qCARACCESS_cate,qLICENSE_cate, qGENDER_cate]

encoder = OrdinalEncoder(categories=categories)
ordinal_enc = encoder.fit_transform(X[ordinal])
ordinal_df = pd.DataFrame(list(map(np.ravel, ordinal_enc)), columns=ordinal,index=X[ordinal].index)

nomial_df = pd.get_dummies(X[oneHot])

X_enc = pd.concat([ordinal_df,nomial_df],axis=1)


#### for targe variables
oe = OrdinalEncoder()
y_multi_enc = oe.fit_transform(y_multi)
y_enc = pd.DataFrame(list(map(np.ravel, y_multi_enc)), columns=alter_col,index=y_multi.index)

y_enc

,Walk,Train/Subway,Bus,Ferry,Access-A-Ride
0,1.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...
1539,1.0,0.0,0.0,0.0,0.0
1540,0.0,1.0,0.0,0.0,0.0
1541,0.0,0.0,0.0,0.0,0.0
1542,0.0,0.0,0.0,0.0,0.0


loading the dataset

In [22]:
y_multi_enc = y_enc.iloc[:,y_enc.columns.isin(['Walk','Train/Subway','Bus'])]
### Oversampling on walk and bus, keep original record in train/subway

In [23]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_multi_enc, test_size=0.2, random_state=1)
# summarize
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (1235, 67) (1235, 3)
Test (309, 67) (309, 3)


## MlKNN

In [39]:
classifier = MLkNN(k=3)
classifier.fit(np.array(X_train) ,np.array(y_train))
preds = classifier.predict(np.array(X_test))
loss = hamming_loss(y_test , preds)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=3 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [41]:
classifier.score(np.array(X_train),y_train)

0.6032388663967612

In [40]:
classifier.score(np.array(X_test),y_test)

0.3851132686084142

## Classifier Chain

In [34]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
multi_target_forest.fit(X_train, y_train)
multi_target_forest.score(X_test,y_test)

0.49838187702265374

In [37]:
# using classifier chains

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(forest)
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions.toarray()))

Accuracy =  0.49838187702265374
